# **<span style="color: #0098cd;">Proyecto: Named-Entity Recognition</span>**


## **Objetivos** 

## **Descripción**

### **Carga y preprocesamiento del texto**

In [38]:
#!pip install spacy
#!pip install chardet
#python -m spacy download es_core_news_md

import pathlib 
import spacy
import pandas as pd
from spacy import displacy
import csv
import chardet

#### **Carga del modelo y detección de codificación**

In [39]:
try:
    nlp = spacy.load("es_core_news_md")  # Cargo el modelo de Spacy para español
except OSError:
    print("El modelo 'es_core_news_md' no está instalado.")

In [40]:
filename = "Dataset.csv"

with open(filename, "rb") as f:
    rawdata = f.read(10000)           # Leo los primeros 10.000 bytes
    result = chardet.detect(rawdata)  # Detecto la codificación
    print(result)                     # Muestro la codificación detectada

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


#### **Carga del dataset**

In [41]:
data = pd.read_csv(filename, delimiter=';', encoding='utf-8', encoding_errors='ignore', low_memory=False)  # Cargo el archivo CSV ignorando carácteres raros.

El dataset tiene una codificación UTF-8 con un 99% de confianza, pero contiene algunos carácteres extraños que representan una cantidad despreciable de carácteres, decido ignorarlos.

### **Primera parte de preguntas**

Preguntas a responder:
1. ¿Cuántos registros contiene el corpus?
2. ¿Cuántas palabras totales hay en los comentarios del corpus?
3. ¿Cuál el número promedio de palabras en cada comentario?

#### **<span style="color: #0098cd;">1. ¿Cuántos registros contiene el corpus? ¿Hay nulos?</span>**

In [42]:
regTotal = data.shape[0]        # Obtengo el número de registros del dataset
#regTotal = len(data)           # Otra opción para obtener el número de registros
regNulos = data.isnull().sum()  # Obtengo el número de nulos en cada columna

print(f"Número de registros en total: {regTotal}")
print("_"*37)
print(f"Valores nulos por columna: ")
print(regNulos)
print("_"*40)

Número de registros en total: 574915
_____________________________________
Valores nulos por columna: 
MEDIO                        0
SOPORTE                     34
URL                        148
TIPO DE MENSAJE            152
CONTENIDO A ANALIZAR       152
INTENSIDAD                 273
TIPO DE ODIO            562619
TONO HUMORISTICO        574769
MODIFICADOR             574562
Unnamed: 9              574912
Unnamed: 10             574912
Unnamed: 11             574913
Unnamed: 12             574913
Unnamed: 13             574913
Unnamed: 14             574913
Unnamed: 15             574913
dtype: int64
________________________________________


In [43]:
data = data.drop(columns=['Unnamed: 9', 'Unnamed: 10',         # Elimino columnas innecesarias con mayoría de nulos
                          'Unnamed: 11', 'Unnamed: 12', 
                          'Unnamed: 13', 'Unnamed: 14', 
                          'Unnamed: 15', 'TONO HUMORISTICO', 
                          'MODIFICADOR', 'TIPO DE ODIO'])


regNoNulos = data.dropna(subset=['CONTENIDO A ANALIZAR', 'INTENSIDAD']) # Elimino los registros con nulos en las columnas 'CONTENIDO A ANALIZAR' e 'INTENSIDAD'
print(f"Registros tras filtrar los nulos: {len(regNoNulos)}")
print("_"*40)

regNulos = data.isnull().sum()              # Compruebo el número de nulos en cada columna de nuevo
print(f"Valores nulos por columna: ")
print(regNulos)

Registros tras filtrar los nulos: 574642
________________________________________
Valores nulos por columna: 
MEDIO                     0
SOPORTE                  34
URL                     148
TIPO DE MENSAJE         152
CONTENIDO A ANALIZAR    152
INTENSIDAD              273
dtype: int64


Elimino las columnas con mayoría de nulos ya que no aportarán información relevante y dado que el análisis se basa en el contenido de los mensajes y su "intensidad" de odio, elimino también los registros donde faltan estos valores.

#### **<span style="color: #0098cd;">2. ¿Cuántas palabras totales hay en los comentarios del corpus?</span>**

In [44]:
# Selecciono una muestra de 15.000 registros aleatorios para pruebas más rápidas
lines_number = 15000                                      # Para cargar solo un número limitado de líneas
data_sample = data.sample(lines_number, random_state=7)   # Tomo una muestra para pruebas más rápidas

print(f"Registros en la muestra: {len(data_sample)}")     # Verifico que se han cogido los 20000 registros.
print("-"*30)

data_sample = data_sample.dropna()                        # Elimino todos los nulos de la muestra

regNulos = data_sample.isnull().sum()                     # Compruebo el número de nulos en la muestra
print(f"Valores nulos por columna en la muestra: ")
print(regNulos)

Registros en la muestra: 15000
------------------------------
Valores nulos por columna en la muestra: 
MEDIO                   0
SOPORTE                 0
URL                     0
TIPO DE MENSAJE         0
CONTENIDO A ANALIZAR    0
INTENSIDAD              0
dtype: int64


In [45]:
# Función para contar las palabras sin signos de puntuación
#----------------------------------------------------------
# is_alpha verifica que sea alfabético, es decir, que no sean signos de puntuación como -> . , ; : ! ? etc.
# is_punct verifica que es un signo de puntuación
# is_space verifica que es un espacio
# Por tanto, si el token es alfabético y no es un signo de puntuación ni un espacio, lo considero una palabra.
def CuentaPalab(mensaje):
    doc = nlp(mensaje)             # Proceso el mensaje con el modelo de SpaCy
    palabras = [token.text for token in doc if token.is_alpha and not token.is_punct and not token.is_space]
    return len(palabras)

# Ejemplo de uso de la función CuentaPalab
ejemplo = "Hola,  ¿cómo estás?   ¿Todo     bien? ¿Todo correcto?."
print("Conteo con SpaCy:", CuentaPalab(ejemplo)) # Debería dar 7

Conteo con SpaCy: 7


In [46]:
# Cuento las palabras teniendo en cuenta los signos de puntuación
ComentMuestra = data_sample[data_sample["TIPO DE MENSAJE"] == "COMENTARIO"]               # Cojo solo los que cuyo TIPO DE MENSAJE es COMENTARIO
palabTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(lambda x: len(str(x).split()))   # Cuento el número total de palabras en los comentarios
SumapalabTotal = palabTotal.sum()
print(f"Palabras en total (contando todo) en comentarios: {SumapalabTotal}")
print("-"*51)

# Cuento las palabras sin tener en cuenta los signos de puntuación ni espacios
AlfTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab).sum()                 # Aplico la función a cada comentario y voy sumando
print(f"Palabras en total (solo alfabéticas) en los comentarios de la muestra: {AlfTotal}")
print("-"*77)


Palabras en total (contando todo) en comentarios: 395990
---------------------------------------------------
Palabras en total (solo alfabéticas) en los comentarios de la muestra: 381987
-----------------------------------------------------------------------------


Debido a los tiempos de procesamiento y a la carga computacional, para esta pregunta y para las posteriores, he seleccionado una muestra de 20.000 registros añadiendo un estado fijo para asegurar que siempre escoja los mismos registros en posteriores ejecuciones y así no distorsionar los resultados.

He considerado que los signos de puntuación no son palabras.

#### **<span style="color: #0098cd;">3. ¿Cuál el número promedio de palabras en cada comentario?</span>**

In [47]:
PalabrasPorComentario = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab) 

AlfProm = PalabrasPorComentario.mean()           # Calculo el promedio
print(f"Promedio de palabras alfabéticas por comentario de la muestra: {AlfProm:.2f}")

Promedio de palabras alfabéticas por comentario de la muestra: 44.29


### **Segunda parte de preguntas**

Considerando dos grupos de comentarios (odio y no odio):

#### **<span style="color: #0098cd;">4. ¿Cuál es el número promedio de palabras en los comentarios de cada grupo?</span>**

In [48]:
# Los valores de INTENSIDAD están almacenados como strings o float en el dataset, así que los convierto a enteros.
data_sample['INTENSIDAD'] = data_sample['INTENSIDAD'].astype(float).astype(int) # Convierto 'INTENSIDAD' primero a float y luego a int

# Separo los comentarios en 2 grupos: los que tienen odio y los que no
ComentNoOdio = data_sample[data_sample["INTENSIDAD"] == 0]  # Selecciono solo los comentarios sin odio (INTENSIDAD == 0)
ComentOdio = data_sample[data_sample["INTENSIDAD"] > 0]     # Selecciono solo los comentarios con odio (INTENSIDAD > 0)

# Calculo el promedio de palabras que hay en los comentarios considerados sin odio
PalabTotalNoOdio = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaPalab) 
PalabPromNoOdio = PalabTotalNoOdio.mean()
print(f"Promedio de palabras alfabéticas en comentarios sin odio: {PalabPromNoOdio:.2f}")

# Calculo el promedio de palabras que hay en los comentarios considerados con odio
PalabTotalOdio = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaPalab)
PalabPromOdio = PalabTotalOdio.mean()
print(f"Promedio de palabras alfabéticas en comentarios con odio: {PalabPromOdio:.2f}")

Promedio de palabras alfabéticas en comentarios sin odio: 107.51
Promedio de palabras alfabéticas en comentarios con odio: 15.74


#### **<span style="color: #0098cd;">5. ¿Cuál es el número promedio de oraciones en los comentarios de cada grupo?</span>**

In [49]:
# Función para contar las oraciones
#----------------------------------
def CuentaOraciones(mensaje):
    doc = nlp(mensaje)             # Proceso el mensaje con el modelo de SpaCy
    oraciones = list(doc.sents)    # Divido el texto en oraciones usando la segmentación de Spacy
    return len(oraciones)


# Ejemplo de uso de la función CuentaOraciones
ejemplo = "Hola. ¿Cómo estás? Todo bien. Espero que tengas un buen día."
print("Conteo con SpaCy:", CuentaOraciones(ejemplo)) # Debería dar 4

Conteo con SpaCy: 4


In [50]:
#Calculo el promedio de oraciones que hay en los comentarios considerados sin odio
OracionTotalNoOdio = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaOraciones)
OracionPromNoOdio = OracionTotalNoOdio.mean()
print(f"Promedio de oraciones en comentarios sin odio: {OracionPromNoOdio:.2f}")

#Calculo el promedio de oraciones que hay en los comentarios considerados con odio
OracionTotalOdio = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaOraciones)
OracionPromOdio = OracionTotalOdio.mean()
print(f"Promedio de oraciones en comentarios con odio: {OracionPromOdio:.2f}")

Promedio de oraciones en comentarios sin odio: 4.00
Promedio de oraciones en comentarios con odio: 1.49


#### **<span style="color: #0098cd;">6. ¿Cuál es el porcentaje de comentarios que contienen entidades NER en cada grupo?</span>**

NER (Named Entity Recognition, Reconocimiento de Entidades Nombradas) es una técnica de NLP utilizada para identificar y clasificar palabras o frases específicas dentro de un texto. Estas palabras representan entidades con significado propio, como nombres de personas, lugares, organizaciones, fechas, cantidades, etc..

Por ejemplo: 
**"Elon Musk es el CEO de Tesla, una empresa con sede en California, y su fortuna supera los 200 mil millones de dólares."**

| **Entidad**                     | **Categoría (NER)**             |
|---------------------------------|--------------------------------|
| Elon Musk                       | **PERSON** (Persona)           |
| Tesla                           | **ORG** (Organización)         |
| California                      | **GPE** (Ubicación geopolítica) |
| 200 mil millones de dólares     | **MONEY** (Dinero)             |


In [51]:
# Ejemplo de funcionamiento
ejemplo = "Lionel Messi juega en el Inter de Miami, un club de fútbol en Estados Unidos."

doc = nlp(ejemplo)          # Proceso el mensaje con el modelo de SpaCy
for entidad in doc.ents:    # Itero sobre las entidades detectadas
    print(f"Entidad: {entidad.text} - Tipo: {entidad.label_}")

Entidad: Lionel Messi - Tipo: PER
Entidad: Inter de Miami - Tipo: ORG
Entidad: en Estados Unidos - Tipo: LOC


In [52]:
# Función para contar las entidades
#----------------------------------
def TieneEntidades(mensaje):
    doc = nlp(mensaje)         # Proceso el mensaje con el modelo de SpaCy
    return len(doc.ents) > 0   # True si al menos hay una entidad

In [54]:
# Creo copias para evitar modificar el dataframe original
ComentNoOdio = ComentNoOdio.copy()
ComentOdio = ComentOdio.copy()

# Calculo el porcentaje de entidades NER en comentarios sin odio
ComentNoOdio['tiene_entidades'] = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(TieneEntidades)    # Aplico la función a cada comentario
PorcEntidadesNoOdio = 100 * ComentNoOdio['tiene_entidades'].mean()                              # Calculo el porcentaje de comentarios sin odio y con entidades NER
print(f"Porcentaje de comentarios sin odio y con entidades NER: {PorcEntidadesNoOdio:.2f}%")

# Calculo el porcentaje de entidades NER en comentarios con odio
ComentOdio['tiene_entidades'] = ComentOdio["CONTENIDO A ANALIZAR"].apply(TieneEntidades)         # Aplico la función a cada comentario
PorcEntidadesOdio = 100 * ComentOdio['tiene_entidades'].mean()                                   # Calculo el porcentaje de comentarios con odio y con entidades NER
print(f"Porcentaje de comentarios con odio y con entidades NER: {PorcEntidadesOdio:.2f}%")

Porcentaje de comentarios sin odio con entidades NER: 59.67%
Porcentaje de comentarios con odio con entidades NER: 38.54%


#### **<span style="color: #0098cd;">7. ¿Cuál es el porcentaje de comentarios que contienen entidades NER de tipo PERSON en cada grupo?</span>**

#### **<span style="color: #0098cd;">8. ¿Cuál es el porcentaje de palabras en cada combinación posible de género y número en cada grupo?</span>**